In [1]:
from scipy.spatial import distance as dist
import cv2
# Load a cascade classifier for detecting eyes
# add the path for your haarcascade_eye.xml file
eye_cascade = cv2.CascadeClassifier('/home/vandit/Downloads/Driver-Snoozing-System/haarcascade_eye.xml')


def Eye_Aspect_Ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear


In [ ]:
threshold=0.22
# Load a cascade classifier for detecting eyes
# eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')

# Initialize the video capture object
cap = cv2.VideoCapture(0)

# Set the frame width and height
cap.set(3, 640)
cap.set(4, 480)

while True:
    # Capture a frame from the camera
    ret, frame = cap.read()

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect eyes in the frame
    eyes = eye_cascade.detectMultiScale(gray)

    # Initialize variables to store the coordinates of the left and right eyes
    left_eye = None
    right_eye = None

    # Iterate over the detected eyes
    for (x, y, w, h) in eyes:
        if x < frame.shape[1] / 2:
            left_eye = (x, y, w, h)
        else:
            right_eye = (x, y, w, h)

    # Check if the left eye is closed
    if left_eye is not None:
        x, y, w, h = left_eye
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        left_ear = Eye_Aspect_Ratio(roi_gray)
        if left_ear < threshold:
            cv2.putText(frame, "Left eye closed", (10,70),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
        else:
            cv2.putText(frame, "Left eye is open", (10,90),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,0,0), 2)
        # Draw a contour around the left eye
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
    else:
         # If left eye is not detected, handle the case here
        pass

    # Check if the right eye is closed
    if right_eye is not None:
        x, y, w, h = right_eye
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        right_ear = Eye_Aspect_Ratio(roi_gray)
        if right_ear < threshold:
            cv2.putText(frame, "Right eye closed", (10,60),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)
        else:
             cv2.putText(frame, "Right eye is open", (10,50),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,0,0), 2)
         # Draw a contour around the right eye
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
    else:
         # If right eye is not detected, handle the case here
        pass


    # Show the frame in a window
    cv2.imshow('Eye Detection', frame)

    # Exit the loop if the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close all windows
cap.release()
cv2.destroyAllWindows()
